# Tank detection YoloV8 model train

This notebook will train a [Yolov8](https://github.com/ultralytics/ultralytics) model for tank detection using publicly available annotated images of tanks.

As the notebook will run the training with `PyTorch`, it is recommended to have GPUs available. If running in Google Colab, go to Edit > Notebook settings and select GPU hardware acceleration.

The first step of the notebook will use [fiftyone](https://github.com/voxel51/fiftyone), to build a dataset of tank images and bounding box annotations for training the computer vision model. The tutorial notebook on [Fine-tuning YOLOv8 models for custom use cases](https://github.com/voxel51/fiftyone/blob/v0.21.0/docs/source/tutorials/yolov8.ipynb) is a usefull introduction on how to use `fiftyone`.

## Setup

In [14]:
%load_ext autoreload
%autoreload 2

To start, check GPU support.

In [1]:
import torch

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.3.0+cu121 (NVIDIA RTX A6000)


We also setup logs.

In [2]:
import logging

logging.basicConfig(level=logging.INFO)

## Download images from ImageNet

The first dataset we'll use is ImageNet21k. The ImageNet21k dataset is available at [https://image-net.org/download-images.php](https://image-net.org/download-images.php). You need to register and be granted access to download the images. We use the Winter 21 version since it gives the option of downloading the images for a single synset: https://image-net.org/data/winter21_whole/SYNSET_ID.tar, e.g., https://image-net.org/data/winter21_whole/n02352591.tar. The processed version of ImageNet21k is available here : https://github.com/Alibaba-MIIL/ImageNet21K. The class ids and names are available here https://github.com/google-research/big_transfer/issues/7#issuecomment-640048775.

We'll begin by downloading the class names that are in ImageNet21k and look for relevant classes that we can use.

In [3]:
from pathlib import Path

imagenet_dir = Path() / "imagenet"

In [4]:
from adomvi.datasets.imagenet import download_class_names, find_class_by_text

classes = download_class_names(imagenet_dir)
find_class_by_text(classes, "military")

INFO:root:File imagenet/imagenet21k_wordnet_ids.txt already exists. Skipping download.
INFO:root:File imagenet/imagenet21k_wordnet_lemmas.txt already exists. Skipping download.


{'n03762982': 'military_hospital',
 'n03763727': 'military_quarters',
 'n03763968': 'military_uniform',
 'n03764276': 'military_vehicle',
 'n04552348': 'warplane, military_plane',
 'n08249459': 'concert_band, military_band',
 'n09809538': 'army_engineer, military_engineer',
 'n09943239': 'commissioned_military_officer',
 'n10316360': 'military_attache',
 'n10316527': 'military_chaplain, padre, Holy_Joe, sky_pilot',
 'n10316862': 'military_leader',
 'n10317007': 'military_officer, officer',
 'n10317500': 'military_policeman, MP',
 'n10512372': 'recruit, military_recruit',
 'n10582746': 'serviceman, military_man, man, military_personnel',
 'n10759331': 'volunteer, military_volunteer, voluntary'}

We can now download images and annotations for the relevant classes. The `download_imagenet_detections` function will download the images and annotations for the given class ids **if the annotations exist** (not all classes have been annotated).

In [5]:
from adomvi.datasets.imagenet import download_imagenet_detections

class_ids = ["n02740300", "n04389033", "n02740533", "n04464852", "n03764276"]
download_imagenet_detections(class_ids, imagenet_dir)

INFO:root:File imagenet/bboxes_annotations.tar.gz already exists. Skipping download.
INFO:root:There are not annotations for class n02740300.
INFO:root:Annotations directory imagenet/labels/n04389033 already exists. Skipping extract.
INFO:root:There are not annotations for class n02740533.
INFO:root:There are not annotations for class n04464852.
INFO:root:There are not annotations for class n03764276.
INFO:root:Deleting annotations dir.


The data we just downloaded into the `imagenet` directory is not all clean: there are annotations which have no corresponding image. We need to remove those labels, otherwise this causes errors when importing the data into fiftyone.

In [6]:
from adomvi.datasets.imagenet import cleanup_labels_without_images

cleanup_labels_without_images(imagenet_dir)

INFO:root:Deleting 0 labels without images


We can now create a new dataset with `fiftyone`. Fiftyone allows us to manage images annotated with bounding boxes and labels, to merge datasets from different sources, and to split the datasets and prepare them for processing.

In [7]:
import fiftyone as fo

# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=imagenet_dir,
    dataset_type=fo.types.VOCDetectionDataset,
)

dataset.name = "military-vehicles"

view = dataset.map_labels(
    "ground_truth",
    {"n04389033":"tank"}
)

 100% |█████████████████| 378/378 [439.9ms elapsed, 0s remaining, 864.4 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 378/378 [439.9ms elapsed, 0s remaining, 864.4 samples/s]      


Once our dataset is created, we can launch a session to display the dataset and view the annotated images

In [8]:
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


INFO:fiftyone.core.session.session:Session launched. Run `session.show()` to open the App in a cell output.


In [9]:
session.open_tab()

<IPython.core.display.Javascript object>

### Add OpenImage samples

The ImageNet dataset only contained 378 annotated images of tanks, so we'll look into other available datasets to improve training of the model. We’ll load [Open Images](https://storage.googleapis.com/openimages/web/index.html) samples with `Tank` detection labels, passing in `only_matching=True` to only load the `Tank` labels. We then map these labels by changing `Tank` into `tank`.

In [10]:
import fiftyone.zoo as foz

oi_samples = foz.load_zoo_dataset(
    "open-images-v7",
    classes = ["Tank"],
    only_matching=True,
    label_types="detections"
).map_labels(
    "ground_truth",
    {"Tank":"tank"}
)

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/home/jrenault/fiftyone/open-images-v7/train' if necessary


Necessary images already downloaded


INFO:fiftyone.utils.openimages:Necessary images already downloaded


Existing download of split 'train' is sufficient


INFO:fiftyone.zoo.datasets:Existing download of split 'train' is sufficient


...

INFO:fiftyone.zoo.datasets:Loading 'open-images-v7' split 'validation'


 100% |███████████████████| 50/50 [74.5ms elapsed, 0s remaining, 671.4 samples/s]     


INFO:eta.core.utils: 100% |███████████████████| 50/50 [74.5ms elapsed, 0s remaining, 671.4 samples/s]     


Dataset 'open-images-v7' created


INFO:fiftyone.zoo.datasets:Dataset 'open-images-v7' created


In [11]:
dataset.merge_samples(oi_samples)

## Fine-tune a YOLOv8 detection model

Now that our dataset is created, we'll export it into a format supported by YOLOv8 to train our model.

In [12]:
import fiftyone.utils.random as four

## delete existing tags to start fresh
dataset.untag_samples(dataset.distinct("tags"))

## split into train, test and val
four.random_split(
    dataset,
    {"train": 0.8, "val": 0.1, "test": 0.1}
)

In [17]:
from pathlib import Path
from adomvi.utils import export_yolo_data

export_dir = Path() / "vehicles"
export_yolo_data(dataset, export_dir, ["tank"], split = ["train", "val", "test"])

  14% |██-------------|  177/1299 [207.1ms elapsed, 1.3s remaining, 854.6 samples/s] 

/home/jrenault/mambaforge/envs/adomvi2/lib/python3.10/site-packages/fiftyone/utils/yolo.py:1029: UserWarning: Ignoring object with label 'n04389033' not in provided classes
  warnings.warn(msg)


 100% |███████████████| 1299/1299 [1.4s elapsed, 0s remaining, 959.7 samples/s]         


INFO:eta.core.utils: 100% |███████████████| 1299/1299 [1.4s elapsed, 0s remaining, 959.7 samples/s]         


Directory 'vehicles' already exists; export will be merged with existing files


 100% |█████████████████| 163/163 [196.8ms elapsed, 0s remaining, 828.2 samples/s]     


INFO:eta.core.utils: 100% |█████████████████| 163/163 [196.8ms elapsed, 0s remaining, 828.2 samples/s]     


Directory 'vehicles' already exists; export will be merged with existing files


 100% |█████████████████| 162/162 [169.1ms elapsed, 0s remaining, 958.2 samples/s]     


INFO:eta.core.utils: 100% |█████████████████| 162/162 [169.1ms elapsed, 0s remaining, 958.2 samples/s]     


In [33]:
from adomvi.yolo.yolo import train

train("yolov10n.pt", data=export_dir / "dataset.yaml")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5.58M/5.58M [00:00<00:00, 11.4MB/s]


AttributeError: Can't get attribute 'YOLOv10DetectionModel' on <module 'ultralytics.nn.tasks' from '/home/jrenault/mambaforge/envs/adomvi2/lib/python3.10/site-packages/ultralytics/nn/tasks.py'>

In [23]:
from pathlib import Path
from adomvi.yolo.yolo import predict

best_model = Path() / "runs/detect/train/weights/best.pt"
results = predict(best_model, source=export_dir / "images/test")


image 1/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/005e6167916abbf0.jpg: 544x640 1 tank, 4.5ms
image 2/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/007377a82dc87b7f.jpg: 448x640 2 tanks, 4.5ms
image 3/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/00741fb6fb4c7209.jpg: 512x640 2 tanks, 4.5ms
image 4/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/00c5be8db0e25f5e.jpg: 448x640 1 tank, 4.4ms
image 5/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/014df43a4a199fe0.jpg: 480x640 15 tanks, 5.2ms
image 6/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/01d793b58ee81477.jpg: 480x640 6 tanks, 4.1ms
image 7/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/02a2b9d0a27104cb.jpg: 480x640 1 tank, 4.0ms
image 8/162 /home/jrenault/workspace/adomvi2/notebooks/vehicles/images/test/0410130b4a7e92bb.jpg: 480x640 1 tank, 4.1ms
image 9/162 /home/jrenault/workspa

In [31]:
results[0].save_txt(Path(results[0].path).with_suffix('.txt'))